# Importação das bibliotecas necessárias

In [1]:
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  classification_report
from sklearn.feature_selection import SelectPercentile

D:\Downloads2\Anaconda\download\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#imports
from time import time
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC
#from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import optuna
from sklearn.metrics import f1_score

In [3]:

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
import nltk
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import RFECV, RFE, SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import set_config
set_config(display='diagram')
from sklearn.svm import SVC
import datetime

# Importação dos dados

In [4]:
data = pd.read_excel('../data/(CORRIGIDO)ep2_pln_train.xlsx')

In [5]:
data.head() # visualização das primeiras 5 linhas do dataframe

,req_text,age
0,- Quantos empregados em cada um dos atuais nív...,a2
1,- Solicito cópia das Atas do Conselho de Admin...,a2
2,"- Solicito informar a norma (lei, decreto, por...",a2
3,"- Solicito por gentileza, a informação sobre a...",a2
4,"- Solicito por gentileza, a informação sobre a...",a2


In [6]:
data.shape # visualização do formato do dataframe

(8200, 2)

In [7]:
data.age.value_counts()

age
a3    3000
a2    2000
a4    2000
a1    1200
Name: count, dtype: int64

In [8]:
data['age'] = data['age'].map({
                                'a4':3,
                                'a3': 2,
                                'a2': 1,
                                'a1': 0})

In [9]:
data.head()

,req_text,age
0,- Quantos empregados em cada um dos atuais nív...,1
1,- Solicito cópia das Atas do Conselho de Admin...,1
2,"- Solicito informar a norma (lei, decreto, por...",1
3,"- Solicito por gentileza, a informação sobre a...",1
4,"- Solicito por gentileza, a informação sobre a...",1


In [10]:
# a seguir os dados serão divididos entre features (X) e label (y)
X = data['req_text'] # features
y = data['age'] # label

# Seleção de hiperparâmetros

Nesta seção, será feita a comparação entre classificadores com os melhores hiperparâmetros encontrados através da utilização do otimizador Optuna, que, resumidamente, funciona de X forma. Os classificadores comparados são:
- Regressão Logística
- Random Forest
- XGBoost

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,
                                                   stratify=y)

# Seleção do modelo

In [13]:
def seleciona_grid(model):

    param_grid = None

    if isinstance(model, MultinomialNB):
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__percentile": [33, 66, 100],
            "estimator__alpha": [50, 15, 10, 5, 1, 0.5, 0.3, 0.1, 0.05, 0.03, 0.02, 0.01,  0.001],
            "estimator__fit_prior": [True, False],
            }

    if isinstance(model, SVC):
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__percentile": [33, 66, 100],
            "estimator__gamma": [1, 0.1, 0.01, 0.001],
            "estimator__kernel": ['linear', 'sigmoid'],
            "estimator__C": [0.1, 1, 10, 100]
            }
            
    if isinstance(model, LinearSVC):
            param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__percentile": [33, 66, 100],
            "estimator__dual": [True, False],
            "estimator__penalty": ['l1', 'l2'],
            "estimator__fit_intercept": [True, False],
            "estimator__C": uniform(loc=0, scale=4)
            }


    if isinstance(model, RandomForestClassifier):
        param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'], 
            "selection__percentile": [33, 66, 100],
            "estimator__n_estimators": np.arange(20,150), 
            "estimator__max_features": ['log2', 'sqrt'],
            "estimator__max_depth": np.arange(10,110),
            "estimator__min_samples_split": np.arange(2,11),
            "estimator__min_samples_leaf": np.arange(1,5),
            "estimator__bootstrap": [True, False]
            }
        
    if isinstance(model, XGBClassifier):
        param_grid = {
            "vect__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
            "vect__analyzer": ['word','char'],
            "selection__percentile": [33, 66, 100],
            "estimator__gamma": np.linspace(0,9,100, dtype=np.int64),
            "estimator__alpha": np.linspace(0,40,100, dtype=np.int64),
            "estimator__lambda": np.linspace(0,3,10, dtype=np.int64),
            "estimator__colsample_bytree": np.linspace(0.2,1,10, dtype=np.int64)
            }

    return param_grid

In [14]:
def fit_e_avalia(random_search):
    
    inicio_random_search = datetime.datetime.now()

    model_trained = random_search.fit(X_train, y_train) # fit

    fim_random_search = datetime.datetime.now()
    tempo_total = fim_random_search - inicio_random_search
    print(f'Duração da Random Search: {tempo_total}')

    y_pred = model_trained.predict(X_test) # predicao

    # Predição F1 e Class Report
    f1 = f1_score(y_test, y_pred, average= 'macro') # f1
    f1 *= 100
    f1 = round(f1,2)
    report = classification_report(y_test, y_pred, output_dict=True) # class report
    
    return model_trained, tempo_total, f1, report

    #######
    
    #if split == 'strat_vies': # estratificacao pela label
    #    model_trained = random_search.fit(X_train_strat_vies, y_train_strat_vies) # fit

    #    fim_random_search = datetime.datetime.now()
    #    tempo_total = fim_random_search - inicio_random_search
    #    print(f'Duração da Random Search: {tempo_total}')

    #    y_pred = model_trained.predict(X_test_strat_vies) # predicao
    #    f1 = f1_score(y_test_strat_vies, y_pred, average= 'macro') # f1
    #    report = classification_report(y_test_strat_vies, y_pred, output_dict=True) # class report

    #elif split == 'strat_partido': # estratificacao pelos partidos 
    #    model_trained = random_search.fit(X_train_strat_part, y_train_strat_part) # fit

    #   fim_random_search = datetime.datetime.now()
    #    tempo_total = fim_random_search - inicio_random_search
    #    print(f'Duração da Random Search: {tempo_total}')

    #    y_pred = model_trained.predict(X_test_strat_part) # predicao
    #    f1 = f1_score(y_test_strat_part, y_pred, average= 'macro') # f1
    #    report = classification_report(y_test_strat_part, y_pred, output_dict=True) # class report

    #elif split == 'pred_partido_novo': # predicao de partidos nao vistos no teste
     #   model_trained = random_search.fit(X_train_part_novos, y_train_part_novos) # fit

     #   fim_random_search = datetime.datetime.now()
     #   tempo_total = fim_random_search - inicio_random_search
     #   print(f'Duração da Random Search: {tempo_total}')

     #   y_pred = model_trained.predict(X_test_part_novos) # predicao
     #   f1 = f1_score(y_test_part_novos, y_pred, average= 'macro') # f1
     #   report = classification_report(y_test_part_novos, y_pred, output_dict=True) # class report
    
    #return model_trained, tempo_total, f1, report

In [15]:
def compara(iteracoes, modelos, nome_arquivo):

    # seletor de features
    selection = SelectPercentile()

    # possibilidades de oversampling ou nao
    samplers = [RandomOverSampler(random_state=42), None]

    # diferentes splits que serao avaliador
    #splits = ['strat_vies', 'strat_partido', 'pred_partido_novo'] 

    # dataframe em que sera inserido os dados do modelo testado 
    df_resultados = pd.DataFrame(columns=['modelo', 'vect','sampler', 'scaling',
                                          'duracao_random_search','qnt_iteracoes',
                                          'f1_randsearch', 
                                           'f1_pred',
                                          'class_report',
                                         'duracao_aval_iv',
                                          'acc_aval_iv',
                                         'melhores_parametros'])
    
    for model in modelos:

        for sampler in samplers:

            #for split in splits:
                
            # seleciona grid de parametros
            param_grid = seleciona_grid(model)
            
            
            scaler = MaxAbsScaler()
    
            # define o pipeline
            pipeline = Pipeline([
                    ('vect', TfidfVectorizer()),
                    ('scaling', scaler), 
                    ('selection', selection),
                    ('ros', sampler),
                    ('estimator', model)
                    ])
    
            
            #  --- Prints das configurações dessa iteracao ---
            print(f'Modelo: {model}')
            #print(f'Split: {split}')
            print(f'Scaler: {scaler}')
            print(f'Sampler: {sampler}')
                
    
            # definicao da randomized search
            random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,cv=StratifiedKFold(n_splits=5),
                                                n_iter=iteracoes, n_jobs=1, random_state=42, scoring='f1_macro')


            # fit e avaliacao pela randomized search
            model_trained, tempo_total, f1, report = fit_e_avalia(random_search)
                
            print('---')
            resultados = model_trained.cv_results_

            for params, score in zip(resultados['params'], resultados['mean_test_score']):
                print(f"Parâmetros: {params}, Score: {score}")
            print('---')    
                
            # melhor metrica na random search
            score_random_search = model_trained.best_score_
            score_random_search *= 100
            score_random_search = round(score_random_search,2)
            print(f'Melhor F1 na Random Search: {score_random_search}%')
            
            # melhores parametros encontrados
            print('Melhores parâmetros encontrados:')
            print(model_trained.best_params_)

            
            # acuracia da predicao
            print(f'F1 macro = {f1}%')
    
            # classification report
            print(report)

            # Avaliação Ivandre
            pipeline = Pipeline([
                            ('vect', TfidfVectorizer()),
                            ('scaling', scaler), 
                            ('selection', selection),
                            ('ros', sampler),
                            ('estimator', model)
                            ])
        
            pipeline = pipeline.set_params(**model_trained.best_params_)
        
            print(f'get_params: {pipeline.get_params}')
            
            inicio_aval_iv = datetime.datetime.now()
            acc_iv = cross_val_score(pipeline, X, y, scoring='accuracy', cv=10, n_jobs=2).mean()
            acc_iv *= 100
            acc_iv = round(acc_iv,2)
            fim_aval_iv = datetime.datetime.now()
            tempo_aval_iv = fim_aval_iv - inicio_aval_iv
            print(f'Duração da Avaliação Ivandre: {tempo_aval_iv}')
        
            print(f'Acurácia Ivandre = {acc_iv}%')
                    
            
            print('----------------------------------------------')
            
            # --- Escrita em memória secundária ---

            # Nova linha que sera adicionada
            nova_linha = {'modelo': model, 'vect': TfidfVectorizer(),
                          'sampler': str(sampler), 'scaling': scaler,
                          'duracao_random_search': tempo_total,
                          'qnt_iteracoes': iteracoes,
                          'f1_randsearch': f'{score_random_search}%',
                          'f1_pred': f'{f1}%', 'class_report': report,
                           'duracao_aval_iv': tempo_aval_iv,
                           'acc_aval_iv': f'{acc_iv}%',
                           'melhores_parametros': str(model_trained.best_params_)}
        
            # Cria um novo DataFrame com a nova linha
            nova_linha_resultados = pd.DataFrame([nova_linha])
        
            # Concatena o novo DataFrame com o DataFrame existente
            df_resultados = pd.concat([df_resultados, nova_linha_resultados], ignore_index=True)

            # salvamento do dataframe de resultados apos os testes terem terminado
            df_resultados.to_csv(nome_arquivo, index=False)
    
    
    print('Fim dos testes')

In [16]:
modelos = [MultinomialNB()]

compara(240, modelos, 'compara-nb-240-.csv')

Modelo: MultinomialNB()
Scaler: MaxAbsScaler()
Sampler: RandomOverSampler(random_state=42)
Duração da Random Search: 3:45:58.140558
---
Parâmetros: {'vect__ngram_range': (2, 3), 'vect__analyzer': 'char', 'selection__percentile': 100, 'estimator__fit_prior': True, 'estimator__alpha': 1}, Score: 0.42394854036705676
Parâmetros: {'vect__ngram_range': (2, 4), 'vect__analyzer': 'char', 'selection__percentile': 100, 'estimator__fit_prior': False, 'estimator__alpha': 50}, Score: 0.3645419901703627
Parâmetros: {'vect__ngram_range': (3, 4), 'vect__analyzer': 'word', 'selection__percentile': 100, 'estimator__fit_prior': False, 'estimator__alpha': 10}, Score: 0.44063908842477717
Parâmetros: {'vect__ngram_range': (1, 4), 'vect__analyzer': 'char', 'selection__percentile': 33, 'estimator__fit_prior': True, 'estimator__alpha': 0.03}, Score: 0.42505367545824263
Parâmetros: {'vect__ngram_range': (1, 3), 'vect__analyzer': 'word', 'selection__percentile': 33, 'estimator__fit_prior': False, 'estimator__alp

In [17]:
yyy

NameError: name 'yyy' is not defined

In [ ]:
def compara(iteracoes):
    
    # Escrita no arquivo
    with open('compara-xg-rf.txt', "w") as arquivo:
        pass
    
    modelos = [XGBClassifier(seed=42),RandomForestClassifier(random_state=42)]
    
    vect = [CountVectorizer(analyzer='word'), TfidfVectorizer(analyzer= 'word')]
    
    for model in modelos:
        
        for vectorizer in vect:
    
            param_grid = None
            
            scaler = MaxAbsScaler()

            preprocessor = ColumnTransformer(transformers=[
                                            ('vect_resp_text', vectorizer,'resp_text'), 
                                            ('vect_pos', vectorizer,'pos')])

            pipeline = Pipeline([
                    ('preprocessor', preprocessor), 
                    ('scaling', scaler), 
                    ('estimator', model)
                    ])

            if isinstance(model, MultinomialNB):
                param_grid = {
                "preprocessor__vect_resp_text__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "preprocessor__vect_pos__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "estimator__alpha": [50, 15, 10, 5, 1, 0.5, 0.3, 0.1, 0.05, 0.03, 0.02, 0.01,  0.001],
                "estimator__fit_prior": [True, False],
                }

            if isinstance(model, SVC):
                param_grid = {
                "preprocessor__vect_resp_text__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "preprocessor__vect_pos__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "estimator__gamma": [1, 0.1, 0.01, 0.001, 0.0001],
                "estimator__kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
                "estimator__C": [0.1, 1, 10, 100, 1000]
                }

            if isinstance(model, LogisticRegression):

                if 'l1' in model.get_params()['penalty']:
                    param_grid = {
                    "preprocessor__vect_resp_text__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                    "preprocessor__vect_pos__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                    "estimator__C": np.linspace(0,10,100),
                    "estimator__solver": ['liblinear','saga']
                    }

                elif 'l2' in model.get_params()['penalty']:
                    param_grid = {
                    "preprocessor__vect_resp_text__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                    "preprocessor__vect_pos__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                    "estimator__C": np.linspace(0,10,100),
                    "estimator__solver": ['lbfgs', 'newton-cg', 'newton-cholesky',
                                          'liblinear','saga', 'sag']
                    }

            if isinstance(model, RandomForestClassifier):
                param_grid = {
                "preprocessor__vect_resp_text__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "preprocessor__vect_pos__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "estimator__n_estimators": np.arange(20,150), 
                "estimator__max_features": ['log2', 'sqrt'],
                "estimator__max_depth": np.arange(10,110),
                "estimator__min_samples_split": np.arange(2,11),
                "estimator__min_samples_leaf": np.arange(1,5),
                "estimator__bootstrap": [True, False]
                }
                
            if isinstance(model, XGBClassifier):
                param_grid = {
                "preprocessor__vect_resp_text__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "preprocessor__vect_pos__ngram_range": [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)],
                "estimator__gamma": np.linspace(0,9,100, dtype=np.int64),
                "estimator__alpha": np.linspace(0,40,100, dtype=np.int64),
                "estimator__lambda": np.linspace(0,3,10, dtype=np.int64),
                "estimator__colsample_bytree": np.linspace(0.2,1,10, dtype=np.int64)
                }

                
                
            # Prints do modelo e da vetorização
            if isinstance(model, LogisticRegression):
                if 'l1' in model.get_params()['penalty']:
                    print(f'Modelo: {model} l1')
                else:
                    print(f'Modelo: {model} l2')
            else:
                print(f'Modelo: {model}')
                
            print(f'Vetorizador utilizado: {vectorizer}')
            
            # Random Search
            comeco_random_search = datetime.datetime.now()
            print(f'Começo da Random Search: {comeco_random_search}')
                
            random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid,cv=StratifiedKFold(n_splits=5),
                                                n_iter=iteracoes, n_jobs=2, random_state=24)
            
            model_trained = random_search.fit(X_train, y_train)

            final_random_search = datetime.datetime.now()
            print(f'Final da Random Search: {final_random_search}')
            
            score_random_search = model_trained.best_score_
            score_random_search *= 100
            score_random_search = round(score_random_search,2)
            print(f'Melhor resultado na Random Search: {score_random_search}%')
            
            print('Melhores parâmetros encontrados:')
            print(model_trained.best_params_)
            
            # Predição
            y_pred = model_trained.predict(X_test)
            acc_pred = accuracy_score(y_test, y_pred)
            acc_pred *= 100
            acc_pred = round(acc_pred,2)
            print(f'Acurácia predita = {acc_pred}%')
                    
            # Avaliação Ivandre
            comeco_av_iv = datetime.datetime.now()
            print(f'Começo da Avaliação Ivandre: {comeco_av_iv}')
            acc_iv = cross_val_score(model_trained, X, y, scoring='accuracy', cv=10, n_jobs=2).mean()
            acc_iv *= 100
            acc_iv = round(acc_iv,2)
            final_av_iv = datetime.datetime.now()
            print(f'Final da Avaliação Ivandre: {final_av_iv}')
        
            print(f'Acurácia Ivandre = {acc_iv}%')
            
            print('----------------------------------------------')
            
            # Escrita no arquivo
            with open('compara-xg-rf.txt', "a") as arquivo:
                
                if isinstance(model, LogisticRegression):
                    if 'l1' in model.get_params()['penalty']:
                        arquivo.write(f'Modelo: {model} l1\n')
                    else:
                        arquivo.write(f'Modelo: {model} l2\n')
                else:
                    arquivo.write(f'Modelo: {model}\n')

                arquivo.write(f'Vetorizador utilizado: {vectorizer}\n')
                
                arquivo.write(f'Começo da Random Search: {comeco_random_search}\n')
                arquivo.write(f'Final da Random Search: {final_random_search}\n')
                arquivo.write(f'Melhor resultado na Random Search: {score_random_search}\n')
                arquivo.write('Melhores parâmetros encontrados:\n')
                arquivo.write(str(model_trained.best_params_))
                arquivo.write('\n')
                arquivo.write(f'Acurácia predita = {acc_pred}%\n')
                arquivo.write(f'Começo da Avaliação Ivandre: {comeco_av_iv}\n')
                arquivo.write(f'Final da Avaliação Ivandre: {final_av_iv}\n')
                arquivo.write(f'Acurácia Ivandre = {acc_iv}%\n')
                arquivo.write('----------------------------------------------\n')

In [ ]:
compara(2)